In [1]:
import numpy as np
import pandas as pd
import os

/Users/wsp/.local/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.8' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
hh = pd.read_csv('data/psam_h06.csv', dtype={'RT': 'str', 'SERIALNO': 'str'})
per = pd.read_csv('data/psam_p06.csv',dtype={'RT': 'str', 'SERIALNO': 'str', 'NAICSP': 'str'})

In [6]:
puma_list = [4101,
 4102,
 7701,
 7702,
 7703,
 7704,
 11300,
 6701,
 6702,
 6703,
 6704,
 6705,
 6706,
 6707,
 6708,
 6709,
 6710,
 6711,
 6712,
 4701,
 4702,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 1700,
 9901,
 9902,
 9903,
 9904,
 5301,
 5302,
 5303,
 1301,
 1302,
 1303,
 1304,
 1305,
 1306,
 1307,
 1308,
 9501,
 1309,
 9502,
 9503,
 8501,
 8502,
 8503,
 8504,
 8505,
 8506,
 8507,
 8508,
 8509,
 8510,
 8511,
 8512,
 8513,
 8514,
 7501,
 7502,
 7503,
 7504,
 7505,
 7506,
 7507,
 10100,
 5500,
 8101,
 8102,
 8103,
 8104,
 8105,
 8106,
 6101,
 6102,
 6103,
 9701,
 9702,
 9703,
 8701,
 8702]

print('making copies in memory ...')
hh_update = hh.copy()
per_update = per.copy()


# select puma within the region
hh_update = hh[hh['PUMA'].isin(puma_list)]
per_update = per_update[per_update['PUMA'].isin(puma_list)]

# exclude zero-person household
hh_update= hh_update[hh_update['NP']>0]

making copies in memory ...


In [26]:
per_update['employed'] = 0
per_update.loc[per_update['ESR'].isin([1,2,4,5]), 'employed'] = 1

per_update.employed.value_counts()

0    314622
1    294208
Name: employed, dtype: int64

In [27]:
hh_workers = per_update.groupby('SERIALNO')['employed'].sum().reset_index().set_index('SERIALNO')['employed'].to_dict()
hh_update['hh_workers_from_esr'] = hh_update['SERIALNO'].map(hh_workers)

hh_update.hh_workers_from_esr.value_counts()

1     82775
0     72658
2     69824
3     14763
4      4671
5      1160
6       324
7        98
8        31
9         7
10        4
15        1
16        1
Name: hh_workers_from_esr, dtype: int64

In [29]:
hh_update = pd.merge(
    left = hh_update,
    right= per_update[['SERIALNO','PWGTP','ADJINC']].drop_duplicates(subset=['SERIALNO']).rename(columns = {'ADJINC': 'PADJINC'}),
    how  = "left"
)

hh_update.loc[hh_update.TYPE==3, "WGTP"] = hh_update.PWGTP
hh_update.loc[hh_update.TYPE==3, "ADJINC"] = hh_update.PADJINC
hh_update.loc[hh_update.TYPE==3, "HHT"] = 0
hh_update.loc[hh_update.TYPE==3, "VEH"] = 0
hh_update.drop(columns=["PWGTP", "PADJINC"], inplace=True)

In [30]:
subject_id = '2015000000151'

hh_df = hh_update.loc[hh_update['SERIALNO'] == subject_id]
hh_df = hh_df[['SERIALNO', 'hh_workers_from_esr', 'ADJINC']]
hh_df.head()

,SERIALNO,hh_workers_from_esr,ADJINC
6,2015000000151,0,1080470


In [25]:
hh_workers

{'2015000000009': 1,
 '2015000000015': 1,
 '2015000000096': 2,
 '2015000000100': 2,
 '2015000000109': 1,
 '2015000000113': 0,
 '2015000000151': 0,
 '2015000000255': 3,
 '2015000000279': 1,
 '2015000000322': 0,
 '2015000000367': 3,
 '2015000000376': 0,
 '2015000000418': 1,
 '2015000000428': 0,
 '2015000000450': 0,
 '2015000000493': 2,
 '2015000000497': 0,
 '2015000000498': 1,
 '2015000000528': 0,
 '2015000000558': 1,
 '2015000000574': 0,
 '2015000000575': 1,
 '2015000000602': 2,
 '2015000000607': 0,
 '2015000000643': 5,
 '2015000000665': 2,
 '2015000000703': 2,
 '2015000000740': 4,
 '2015000000773': 1,
 '2015000000781': 0,
 '2015000000799': 0,
 '2015000000809': 1,
 '2015000000822': 1,
 '2015000000890': 1,
 '2015000000898': 0,
 '2015000000912': 1,
 '2015000000966': 2,
 '2015000000967': 3,
 '2015000000988': 2,
 '2015000001010': 3,
 '2015000001146': 1,
 '2015000001176': 2,
 '2015000001192': 1,
 '2015000001194': 1,
 '2015000001252': 0,
 '2015000001345': 3,
 '2015000001389': 5,
 '20150000014